In [1]:
import json
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
def fetch_all(query):
    connection = mysql.connector.connect(host='localhost', database='rncs', user='admin', password='Pg49mkMfF4SuLLbA')
    cursor = connection.cursor(buffered=True)    
    cursor.execute(query)
    full = cursor.fetchall()
    cursor.close()
    connection.close()
    return full

# Extract financial information

In [3]:
sirens = np.load('/project/0_cleaning/output_cleaning/sirens.npy').tolist()

query = f'''
    SELECT
        bilan.siren,
        bilan.id,
        bilan.code_type_bilan,
        bilan.date_cloture_exercice,
        bilan.duree_exercice_n,
        bilan_liasse.page,
        bilan_liasse.code,
        bilan.code_devise,
        bilan_liasse.m1,
        bilan_liasse.m2,
        bilan_liasse.m3,
        bilan_liasse.m4        
    FROM
        bilan_liasse
    LEFT JOIN bilan ON bilan.id = bilan_liasse.id_bilan
    WHERE
        bilan.siren IN {tuple(sirens)}   
'''

result = fetch_all(query)
result_df = pd.DataFrame(result, 
                         columns=['siren', 'id_bilan', 'balance_sheet_type', 'FYE', 'length_period', 'page', 'code', 'currency', 'm1', 'm2', 'm3', 'm4'])

result_df['m1'] = result_df['m1'].astype(float)
result_df['m2'] = result_df['m2'].astype(float)
result_df['m3'] = result_df['m3'].astype(float)
result_df['m4'] = result_df['m4'].astype(float)
result_df['FYE'] = pd.to_datetime(result_df['FYE'])
result_df = result_df.loc[:,['siren', 'id_bilan', 'balance_sheet_type', 'FYE', 'length_period', 'page','code', 'currency', 'm1', 'm2', 'm3', 'm4']]
result_df.to_csv(f'bilan_liasse_for_sirens.csv') 
result_df.head(3)

,siren,id_bilan,balance_sheet_type,FYE,length_period,page,code,currency,m1,m2,m3,m4
0,006580195,1104121,C,2016-12-31,12,01,CX,EUR,591891.0,62439.0,529453.0,206176.0
1,006580195,1104121,C,2016-12-31,12,01,AF,EUR,1258988.0,1139276.0,119712.0,184629.0
2,006580195,1104121,C,2016-12-31,12,01,AJ,EUR,79017.0,NaN,79017.0,64982.0


In [4]:
result_df[result_df['code'] == '096'].head()

,siren,id_bilan,balance_sheet_type,FYE,length_period,page,code,currency,m1,m2,m3,m4
8393,315014019,2360356,S,2017-09-30,12,01,096,EUR,492088.0,10711.0,481377.0,NaN
12227,321919953,665259,S,2016-12-31,12,01,096,EUR,1593677.0,NaN,1593677.0,1426255.0
15319,325997112,779535,S,2016-12-31,12,01,096,EUR,374702.0,NaN,374702.0,NaN
15365,325997112,1234319,S,2016-12-31,12,01,096,EUR,374702.0,NaN,374702.0,NaN
16532,327610226,538440,S,2016-12-31,12,01,096,EUR,214387.0,408.0,213979.0,236483.0


In [5]:
len(result_df['siren'].unique())

1530

In [6]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203602 entries, 0 to 203601
Data columns (total 12 columns):
siren                 203602 non-null object
id_bilan              203602 non-null int64
balance_sheet_type    203602 non-null object
FYE                   203602 non-null datetime64[ns]
length_period         203602 non-null object
page                  203602 non-null object
code                  203602 non-null object
currency              203602 non-null object
m1                    149381 non-null float64
m2                    86308 non-null float64
m3                    100251 non-null float64
m4                    64141 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 18.6+ MB


# Extract financial ratios

In [7]:
def get_accounting_line(df, code, m, expense_true):
    filtered = df.loc[df['code'] == code, m]
    try:
        result = float(filtered)
    except:
        result = np.nan
    else:
        if expense_true:
            result = -result
    return result

In [8]:
ratios = []

for siren in result_df['siren'].unique():
    latest_period = max(result_df.loc[result_df['siren'] == siren]['FYE'].unique())
    df = result_df[(result_df['siren'] == siren) & (result_df['FYE'] == latest_period)]
    type_bilan = df.iloc[0,2]

    if type_bilan == 'S':
        debt = get_accounting_line(df, '176', 'm3', 0)
        LT_debt = get_accounting_line(df, '195', 'm3', 0)
        gross_profit = np.nansum([get_accounting_line(df, '270', 'm1', 0), # resultat exploitation
                               get_accounting_line(df, '280', 'm1', 0), # produits financiers
                               get_accounting_line(df, '290', 'm1', 0), # produits exceptionnels
                               get_accounting_line(df, '294', 'm1', 1), # charges financières
                               get_accounting_line(df, '300', 'm1', 1)]) # charges exceptionnelles
        net_profit = get_accounting_line(df, '310', 'm1', 0)
        equity = get_accounting_line(df, '142', 'm3', 0)
        total_assets = get_accounting_line(df, '110', 'm3', 0)
        revenues = get_accounting_line(df, '232', 'm1', 0)
        current_assets = get_accounting_line(df, '096', 'm3', 0)
        current_liabilities = total_assets - LT_debt - equity
        inventory = np.nansum([get_accounting_line(df, '050', 'm3', 0),
                               get_accounting_line(df, '060', 'm3', 0)])

    else:
        debt = get_accounting_line(df, 'EC', 'm1', 0)
        LT_debt = np.nansum([get_accounting_line(df, 'DS', 'm1', 0),
                            get_accounting_line(df, 'DT', 'm1', 0),
                            get_accounting_line(df, 'DU', 'm1', 0),
                             get_accounting_line(df, 'DV', 'm1', 0)])
        gross_profit = get_accounting_line(df, 'GW', 'm3', 0)
        equity = get_accounting_line(df, 'DL', 'm1', 0)
        total_assets = get_accounting_line(df, 'CO', 'm3', 0)
        revenues = get_accounting_line(df, 'FR', 'm3', 0)
        current_assets = get_accounting_line(df, 'CJ', 'm3', 0)
        current_liabilities = total_assets - LT_debt - equity
        inventory = np.nansum([get_accounting_line(df, 'BL', 'm3', 0),
                               get_accounting_line(df, 'BN', 'm3', 0),
                               get_accounting_line(df, 'BP', 'm3', 0),
                               get_accounting_line(df, 'BR', 'm3', 0),
                               get_accounting_line(df, 'BT', 'm3', 0)])
        if type_bilan == 'C':
            net_profit = get_accounting_line(df, 'DI', 'm1', 0)
        else: # type == K
            net_profit = get_accounting_line(df, 'P2', 'm1', 0)

    ratios.append({
        'siren': siren,
        'balance_sheet_type': type_bilan,
        'length_period': df.iloc[0,4],
        'period': latest_period,
        'debt-to-income': debt/gross_profit,
        'debt-to-equity': debt/equity,
        'LTdebt-to-equity': LT_debt/equity,
        'debt-to-assets': debt/total_assets,
        'financial_leverage': total_assets/equity,
        'profitability': net_profit/revenues,
        'current_ratio': current_assets/current_liabilities,
        'quick_ratio': (current_assets-inventory)/current_liabilities, 
    })
        
ratios_df = pd.DataFrame(ratios)
ratios_df.head()

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in double_scalars
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in double_scalars
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in double_scalars


,LTdebt-to-equity,balance_sheet_type,current_ratio,debt-to-assets,debt-to-equity,debt-to-income,financial_leverage,length_period,period,profitability,quick_ratio,siren
0,0.441311,C,1.344303,0.693462,3.331828,250.759729,4.804630,15,2018-03-31,0.006019,0.732884,006580195
1,0.296338,C,2.727945,0.495044,0.980369,18.703920,1.980369,12,2017-12-31,0.014249,1.684645,007220338
2,3.805500,C,3.432042,0.816981,4.463915,72.897150,5.463915,12,2018-01-31,0.014318,2.697708,015751530
3,0.223805,C,2.262617,0.436549,0.774779,NaN,1.774780,12,2018-06-30,NaN,0.566685,057813677
4,1.751583,C,1.433132,0.745532,2.929773,19.376612,3.929773,12,2018-03-31,0.009911,1.250383,065501850


# Create features

In [9]:
ratios_features = pd.DataFrame(sirens, columns=['siren']).merge(ratios_df, how='left')
ratios_features.drop(columns=['length_period', 'period'], inplace=True)
ratios_features.shape

(3573, 10)

In [10]:
ratios_features.head(20)

,siren,LTdebt-to-equity,balance_sheet_type,current_ratio,debt-to-assets,debt-to-equity,debt-to-income,financial_leverage,profitability,quick_ratio
0,332224393,0.252891,C,1.416382,0.669282,2.023722,8.235874,3.023722,0.039486,1.006016
1,798102158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,414305508,-1.833468,C,2.094002,1.626834,-2.595320,-2.350416,-1.595320,-0.142576,2.094002
3,533910931,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,809944416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,343653234,0.000000,C,0.926413,0.483872,0.937506,2.115235,1.937508,0.073568,0.926413
6,451479174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,763500337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,487901951,NaN,S,NaN,1.298757,-4.347197,29.762673,-3.347197,0.066767,NaN
9,408416360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
ratios_features.to_csv('/project/1_feature_extraction/output_feature_extraction/financial_ratios.csv', index=False)

In [12]:
ratios_features.isna().sum()

siren                    0
LTdebt-to-equity      2350
balance_sheet_type    2043
current_ratio         2351
debt-to-assets        2093
debt-to-equity        2093
debt-to-income        2412
financial_leverage    2091
profitability         2459
quick_ratio           2351
dtype: int64